## Step-by-step implementation plan 

1.  Data collection

    1.  Get tickers from S&P 500 on Wikipedia (see [5 Lines of Python to Automate Getting the S&P 500](https://medium.com/wealthy-bytes/5-lines-of-python-to-automate-getting-the-s-p-500-95a632e5e567) for inspiration)

    2.  Get main outcome: ESG risk, obtained through the yahooquery Pypi package (accessed with the parameter esg_scores for a ticker)

    3.  Obtain search metrics on ESG related keywords from Google trends through the pytrends Pypi package 

3.  Feature transformation

    1.  Collapse time dimension of Google trends search index into the following metrics for each keyword, using a defined time span, such as one year

        1.  Maximum search index 

        2.  Avg. search index 

    3.  Possibly Impute missing with mean

    4.  Split data into a train and test set, convert to .csv and upload to S3

5.  Descriptive statistics

    1.  Pick ten firms and show feature averages in a table

    2.  Correlation matrix of features and outcome

    3.  Create a pairplot

7.  Training, validating and testing a model with Sagemaker

    1.  Write scripts for benchmark linear regression: train.py

    2.  Write scripts for PyTorch neural network: model.py and train.py

    3.  Instantiate estimators with Sagemaker 

    4.  Run training job

    5.  Deploy models for testing

9.  Evaluation and benchmark comparison 

    1.  RMSE and R-squared

    2.  Possible model adjustment of the neural net when it lacks precision

11. Cleanup

    1.  Delete endpoint

    2.  Remove other resources, such as training jobs, endpoint configurations, notebook instances

In [5]:
# install required packages 
!pip install yahooquery

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [6]:
!pip install pytrends

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [7]:
# data preparation
import pandas as pd
import numpy as np
import re 
from math import ceil
from math import exp
import os

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# APIs for collecting data 
from yahooquery import Ticker
from pytrends.request import TrendReq 
from time import sleep # avoid too many requests error 

### General
#### Version info

In [8]:
py_version = !python --version
print(py_version[0])
print("Pandas version",pd.__version__)
print("Pytorch version")

Python 3.6.10 :: Anaconda, Inc.
Pandas version 1.0.3
Pytorch version


#### Sagemaker Setup

In [9]:
# role


# Session
# S3 bucket
# folder prefix

# Gather data

## S&P 500 Listings

In [23]:
# retrieve S&P 500 listings from Wikipedia
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_sp500 = table[0]

print("SHAPE:",df_sp500.shape)

# TICKER
ticker = list(df_sp500.Symbol)
print("\nTICKER EXAMPLE:")
print(ticker[:3])

SHAPE: (505, 9)

TICKER EXAMPLE:
['MMM', 'ABT', 'ABBV']


## Yahooquery: ESG ratings

In [26]:
# init yahoo ticker
tickers = Ticker(ticker)

# obtain ESG data (Yahoo x Sustainalytics)
esg_data = tickers.esg_scores
df_esg_raw = pd.DataFrame(esg_data)

def df_to_numeric(ds):
    """Converts columns to numeric where possible"""
    try: 
        return pd.to_numeric(ds)
    except:
        return ds

## wrangle dataframe
# replace strings missings with np.nan and bools with 0 or 1
# transform s.t. each row represents a firm 
# convert columns to numeric, where feasible
df_esg = df_esg_raw.replace({'No fundamentals data found for any of the summaryTypes=esgScores': np.nan,
                            False: 0, 
                            True: 1})\
                    .T\
                    .apply(df_to_numeric)

# store tickers for missing ESG info 
ticker_missing_esg = df_esg[df_esg.isnull().any(axis=1) == True].index

# drop missings
df_esg.dropna(inplace=True)

print(df_esg.iloc[:,1:5].head())

# main outcome, y:total ESG score 
y = df_esg.totalEsg.values

      totalEsg  environmentScore  socialScore  governanceScore
MMM      34.75             12.79        13.89             8.07
ABT      29.83              2.98        16.21            10.63
ABBV     30.19              0.96        16.53            12.69
ACN      11.24              0.55         4.98             5.71
ATVI     16.71              0.15        10.46             6.09

Focus on totalEsg score:
[34.75 29.83 30.19 11.24 16.71 13.59 12.38 33.15 23.62 16.79 11.24 21.16
 36.72 24.69 36.02 22.63 29.62 24.82 30.57 40.24 30.59 34.92 22.13 32.12
 28.99 27.87 17.78 29.46 20.43 23.81 16.7  23.   32.05 39.05 18.66 23.62
 15.01 34.24 19.35 24.72 26.02 18.71 35.63 12.4  11.99 12.68 18.26 16.2
 27.32 21.49 27.97 31.74 11.85 21.56 23.43 23.19 38.98 19.45 23.99 14.35
 30.79 28.19 26.32 19.34 25.96 49.96 22.   26.55 15.64 13.5  21.72 37.02
 21.69  8.64 29.85 28.69 31.55 25.48 20.1  47.74 26.89 20.   40.22 24.74
 24.82 25.23 26.16 25.35 12.08 28.59 27.92 18.09 23.5  27.97 26.23 21.67
 21.82 19.65 

## Pytrends 

### Helper functions

In [21]:
## inlcude into helper functions
# KEYWORD GENERATOR HELPERS 
def regex_strip_legalname(raw_names):
    """Removes legal entity, technical description or firm type from firm name
    
    Input
        raw_names: list of strings with firm names
        
    Return
        list of strings: firm names without legal description 
    
    """
    
    pattern = r"(,\s)?(LLC|Inc|Corp\w*|\(?Class \w+\)?|Group|Company|\WCo(\s|\.)|plc|Ltd|Int'l\.|Holdings)\.?\W?"
    stripped_names = [re.sub(pattern,'', n) for n in raw_names]
    
    return stripped_names

def batch(lst, n=5):
    """Yield successive n-sized chunks from list lst
    
    adapted from https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    
    Input
        lst: list 
        n: selected batch size
        
    Return 
        List: lst divided into batches of len(lst)/n lists
    """
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def flatten_list(nested_list):
    """Flattens nested list"""
    
    return [element for sublist in nested_list for element in sublist]

def list_remove_duplicates(l):
    """Removes duplicates from list elements whilst preserving element order
    adapted from 
    https://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-whilst-preserving-order
    
    Input
        list with string elements
    
    Return 
        Sorted list without duplicates
    
    """
    seen = set()
    seen_add = seen.add
    return [x for x in l if not (x in seen or seen_add(x))]



# PYTREND HELPERS
def pytrends_sleep_init(seconds):
    """Timeout for certain seconds and re-initialize pytrends
    
    Input
        seconds: int with seconds for timeout
        
    Return
        None
    
    """
    print("TIMEOUT for {} sec.".format(seconds))
    sleep(seconds)
    pt = TrendReq()
    
def make_x_y_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    
    # create dir if nonexistent
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # merge df
    y = pd.DataFrame(y)
    x = pd.DataFrame(x)
    
    # export to csv
    pd.concat([y, x], axis=1).to_csv(os.path.join(data_dir, filename), 
                                     header=False, 
                                     index=False)
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))
    
def make_csv(x, filename, data_dir, append=False, header=False, index=False):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    
    # create dir if nonexistent
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # make sure its a df
    x = pd.DataFrame(x)
    
    # export to csv
    if not append:
        x.to_csv(os.path.join(data_dir, filename), 
                                     header=header, 
                                     index=index)
    else:
        x.to_csv(os.path.join(data_dir, filename),
                                     mode = 'a',
                                     header=header, 
                                     index=index)        
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))

### Engineer search keywords from firm names and topic

**TODO:** def construct_search_keywords()

In [22]:
# FIRM NAMES
firm_names_raw = list(df_sp500.Security)

# remove legal taxonomy and firm type
# TODO: add list_remove_duplicates()
firm_names = regex_strip_legalname(firm_names_raw)

# esg keywords (negative exclusion criteria)
topics = ['scandal', 'greenwashing', 'corruption', 'fraud', 'bribe', 'tax', 'forced', 'harassment', 'violation', 
          'human rights', 'conflict', 'weapons', 'arms trade', 'pollution', 'CO2', 'emission', 'fossil fuel',
          'gender inequality', 'discrimination', 'sexism', 'racist', 'intransparent', 'data privacy', 'lawsuit', 
          'unfair', 'bad', 'problem', 'hate', 'issues', 'controversial']

# store lists as csv for retrieval
make_csv(topics, 'topics.csv', 'data', append=False, header=True)
make_csv(firm_names, 'firm_names.csv', 'data', append=False, header=True)

Path created: data/topics.csv
Path created: data/firm_names.csv


In [60]:
############################
# DEFINE PARAMETERS 
n_firms = 30
batch_size = 5
n_keywords = int(n_firms*len(topics))
n_query = int(n_keywords/batch_size)
n_topics = len(topics)
sec_sleep = 45
############################


# create search keywords as pairwise combintations of firm names + topics
search_keywords = [[j+' '+i for j in topics] for i in firm_names]

# print("{} topic keywords for {} firm each ---> {} pairwise combinations"\
#       .format(n_topics, n_firms, n_keywords))
# print()

# Subset for test purposes
print(">>>>>>> Subset for testing purposes")
keywords_sample = search_keywords[:n_firms]
print("Generated {} keywords for {} firms and {} topics each".format(n_keywords,n_firms,n_topics))
print("Resulting in {} queries with {} keywords each (=batch)".format(n_query, batch_size))

## generate keyword batches (= query)
# flatten list
keyword_batches = flatten_list([list(batch(keywords_sample[i], batch_size)) for i in range(n_firms)])

print("\nExample keyword batch:\n{}".format(keyword_batches[0]))

>>>>>>> Subset for testing purposes
Generated 900 keywords for 30 firms and 30 topics each
Resulting in 180 queries with 5 keywords each (=batch)

Example keyword batch:
['scandal 3M ', 'greenwashing 3M ', 'corruption 3M ', 'fraud 3M ', 'bribe 3M ']


### Query Google

In [61]:
## retrieve Google trends across time

# initialize pytrends
pt = TrendReq()

# store DFs for later concat
df_list = []
index_batch_error = []

# create csv to store intermediate results
make_csv(pd.DataFrame(), filename='googletrends.csv', data_dir='data', append=False)

for i, batch in enumerate(keyword_batches):
    
    # retrieve interest over time
    try:
        # re-init pytrends and wait (sleep/timeout)
        pytrends_sleep_init(sec_sleep)
        
        # pass keywords to pytrends API
        pt.build_payload(kw_list=batch) 
        print("Payload build for {}. batch".format(i))
        df_search_result = pt.interest_over_time()
        
    except Exception as e:
        print(e)
        print("Query {} of {}".format(i, n_query))
        # store index at which error occurred
        index_batch_error.append(i)
        
        # re-init pytrends and wait (sleep/timeout)
        pytrends_sleep_init(sec_sleep)
        
        # retry
        print("RETRY for {}. batch".format(i))
        pt.build_payload(kw_list=batch) 
        df_search_result = pt.interest_over_time()
        
    # check for non-empty df
    if df_search_result.shape[0] != 0:
        
        # reset index for consistency (to call pd.concat later with empty dfs)
        df_search_result.reset_index(inplace=True)
        df_list.append(df_search_result)
        
    # no search result for any keyword
    else:        
        # create df containing 0s
        df_search_result = pd.DataFrame(np.zeros((261,batch_size)), columns=batch)
        df_list.append(df_search_result)
        
    make_csv(df_search_result, filename='googletrends.csv', data_dir='data',
             append=True,
            header=True)

Path created: data/googletrends.csv
Payload build for 0. batch
Path created: data/googletrends.csv
Payload build for 1. batch
Path created: data/googletrends.csv
Payload build for 2. batch
Path created: data/googletrends.csv
Payload build for 3. batch
Path created: data/googletrends.csv
Payload build for 4. batch
Path created: data/googletrends.csv
Payload build for 5. batch
Path created: data/googletrends.csv
Payload build for 6. batch
Path created: data/googletrends.csv
Payload build for 7. batch
Path created: data/googletrends.csv
Payload build for 8. batch
Path created: data/googletrends.csv
Payload build for 9. batch
Path created: data/googletrends.csv
Payload build for 10. batch
Path created: data/googletrends.csv
Payload build for 11. batch
Path created: data/googletrends.csv
Payload build for 12. batch
Path created: data/googletrends.csv
Payload build for 13. batch
Path created: data/googletrends.csv
Payload build for 14. batch
Path created: data/googletrends.csv
Payload build 

Path created: data/googletrends.csv
Payload build for 127. batch
Path created: data/googletrends.csv
Payload build for 128. batch
Path created: data/googletrends.csv
Payload build for 129. batch
Path created: data/googletrends.csv
Payload build for 130. batch
Path created: data/googletrends.csv
Payload build for 131. batch
Path created: data/googletrends.csv
Payload build for 132. batch
Path created: data/googletrends.csv
Payload build for 133. batch
Path created: data/googletrends.csv
Payload build for 134. batch
Path created: data/googletrends.csv
Payload build for 135. batch
Path created: data/googletrends.csv
Payload build for 136. batch
Path created: data/googletrends.csv
Payload build for 137. batch
Path created: data/googletrends.csv
Payload build for 138. batch
Path created: data/googletrends.csv
Payload build for 139. batch
Path created: data/googletrends.csv
Payload build for 140. batch
Path created: data/googletrends.csv
Payload build for 141. batch
Path created: data/google

In [80]:
# combine query results to df
drop_cols = ['isPartial', 'date']

# index df
df_clean_list = []
for i,x in enumerate(range(0,len(df_list),6)):

    map_colnames = dict(zip(search_keywords[i+272], list(topics)))
    
    ## create firm-level df
    # df with isPartial and date columns --> drop columns
    try:
        df_firm = pd.concat(df_list[x:x+6], axis=1).drop(columns=drop_cols)

        # rename columns
        df_firm.rename(columns=map_colnames, inplace=True)
        
        # add firm column
        df_firm['firm'] = firm_names[i+272]
        
        df_clean_list.append(df_firm)
        
    except:
        df_firm = pd.concat(df_list[x:x+6], axis=1).rename(columns=map_colnames)
        # rename columns
        df_firm.rename(columns=map_colnames, inplace=True)
        # add firm 
        df_firm['firm'] = firm_names[i]

        df_clean_list.append(df_firm)

# df (long format) with time dimension      
df_time = pd.concat(df_clean_list)

# Store query results so far
print('Index batch error:',index_batch_error)

# get timestamp
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
df_filename = 'df_time_{}_idxbatch_{}.csv'.format(timestr, 1633)
print(df_filename)

# Store df_time
make_csv(df_time, filename=df_filename, data_dir='data', append=False, header=False)